In [ ]:
import cv2
import numpy as k
import face_recognition
import os 

path = "videos/"
images = []
student_names = []

mylist = os.listdir(path)
name_to_id = {}

def attendance(name, student_id):
    csv_path = "attendance.csv"

    # Check if the student has already been marked present
    with open(csv_path, "r") as file:
        for line in file:
            if name in line:
                return False  # Student already marked present

    # If not, mark the student present and write to the CSV file
    with open(csv_path, "a") as file:
        file.write(f"{name},{student_id}\n")
        print(f"{name} with ID {student_id} marked present.")
        return True

for stu in mylist:
    folderpath = os.path.join(path,stu)
    for img in os.listdir(folderpath):
        if img.endswith('.jpg'):
            curimg = cv2.imread(f'{folderpath}/{img}')
            images.append(curimg)
            # Split name and ID based on the file name format
            name_parts = os.path.splitext(stu)[0].split('_')
            if len(name_parts) == 2:
                name, student_id = name_parts
                student_names.append(name)
                name_to_id[name] = student_id


In [2]:
def encoder(images):
    known_faces = []
    for photo in images:
        enc_photo = face_recognition.face_encodings(photo)[0]
        known_faces.append(enc_photo)
    return known_faces

# Test for the encoder
known = encoder(images)

In [ ]:
import pickle
with open('model.pkl', 'rb') as file:
    known = pickle.load(file)

image_path = "test.jpg"  # Replace with the actual path

# Read the image
image = cv2.imread(image_path)


# Get face locations
face_locationss = face_recognition.face_locations(image)
print(len(face_locationss))
enc_test = face_recognition.face_encodings(image,face_locationss)
for each, loc in zip(enc_test, face_locationss):
    is_match = face_recognition.compare_faces(known, each)
    distance = face_recognition.face_distance(known, each)
    best_match = k.argmin(distance)

    if best_match < len(student_names):
        if is_match[best_match]:
            name = student_names[best_match]
            student_id = name_to_id.get(name, "UNKNOWN")
            print(name, student_id)
            attendance(name, student_id)
            y1, x2, y2, x1 = loc
            #y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(image, (x1, y1), (x2, y2), (120, 255, 10), 2)
            cv2.rectangle(image, (x1, y2 - 25), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(image, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
    else:
        print("Invalid best_match index.")

# Draw rectangles around detected faces
for face_location in face_locationss:
    top, right, bottom, left = face_location
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)


# Display the image with rectangles
new = cv2.resize(image,(0,0),None,0.25,0.25)
cv2.imshow('Detected Faces', new)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(known, file)